<a href="https://colab.research.google.com/github/ranjith13119/ANN/blob/main/Cifer_with_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [ ]:
(train_images,train_labels),(test_images,test_labels)=cifar10.load_data()

In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(32, 32, 3)

In [ ]:
train_images=train_images.reshape(len(train_images),32,32,3)
test_images=test_images.reshape(len(test_images),32,32,3)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
                            
    # Input Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        kernel_initializer='he_uniform',
        padding = 'same',
        input_shape=(32,32,3)
    ),
    keras.layers.BatchNormalization(),

     # First Conv. Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        kernel_initializer='he_uniform',
        padding = 'same',
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.2),

    # Second Conv. Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
        kernel_initializer='he_uniform',
        padding = 'same',
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),

    # Third Conv. Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_4_filter', min_value=64, max_value=128, step=16),
        kernel_size=hp.Choice('conv_4_kernel', values = [3,5]),
        kernel_initializer='he_uniform',
        padding = 'same',
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),

    # Four Conv. Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_5_filter', min_value=128, max_value=256, step=16),
        kernel_size=hp.Choice('conv_5_kernel', values = [3,5]),
        kernel_initializer='he_uniform',
        padding = 'same',
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),

    # Five Conv. Layer
    keras.layers.Conv2D(
        filters=hp.Int('conv_6_filter', min_value=128, max_value=256, step=16),
        kernel_size=hp.Choice('conv_6_kernel', values = [3,5]),
        kernel_initializer='he_uniform',
        padding = 'same',
        activation='relu'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.4),

    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu', kernel_initializer='he_uniform'
    ),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output_1',project_name="Cifer Dataset")

INFO:tensorflow:Reloading Oracle from existing project output_1/Cifer Dataset/oracle.json
INFO:tensorflow:Reloading Tuner from output_1/Cifer Dataset/tuner0.json


In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 80)        6

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(monitor='val_acc',verbose=1,min_delta=0.001,patience=3,baseline=0.99)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop, learning_rate_reduction]

In [ ]:
model_result = model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3, callbacks = callbacks)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(model_result.history['loss'], label='train loss')
plt.plot(model_result.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(model_result.history['accuracy'], label='train acc')
plt.plot(model_result.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')